In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [3]:
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [4]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.
/home/tinex/newAnaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
regressor = Sequential()


regressor.add(LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 64, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 64, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 64))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

In [7]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs = 20, batch_size = 32)

Epoch 1/20
1198/1198 [==============================] - 27s 22ms/step - loss: 0.0432
Epoch 2/20
1198/1198 [==============================] - 26s 22ms/step - loss: 0.0056
Epoch 3/20
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0046
Epoch 4/20
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0046
Epoch 5/20
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0045
Epoch 6/20
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0040
Epoch 7/20
1198/1198 [==============================] - 19s 16ms/step - loss: 0.0042
Epoch 8/20
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0056
Epoch 9/20
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0040
Epoch 10/20
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0040
Epoch 11/20
1198/1198 [==============================] - 22s 18ms/step - loss: 0.0035
Epoch 12/20
1198/1198 [==============================] - 28s 24

In [ ]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()